# Голосовой помощник
## Демонстрационный пайплайн использования акустических моделей.

### Установка переменных среды и импорт библиотек

In [1]:
import os
os.environ['HF_HOME'] = '/home/jovyan/work/HF_cache/'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1, 2"

In [2]:
import nemo.collections.asr as nemo_asr
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from sbert_punc_case_ru import SbertPuncCase
from peft import PeftModel, PeftConfig
from TeraTTS import TTS
from ruaccent import RUAccent
import pickle

In [3]:
device_0 = torch.device('cuda:0')
device_1 = torch.device('cuda:1')

### Загрузка моделей

In [4]:
# Модель для распознавания речи nvidia/stt_ru_conformer_transducer_large
asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_ru_conformer_transducer_large")
asr_model = asr_model.to(device_0)

[NeMo I 2023-12-11 08:36:06 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-12-11 08:36:07 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-12-11 08:36:07 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: fals

[NeMo I 2023-12-11 08:36:07 features:289] PADDING: 0


[NeMo W 2023-12-11 08:36:09 nemo_logging:349] /home/jovyan/.cache/pypoetry/virtualenvs/speech-pipe-aX1LTVkJ-py3.10/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-12-11 08:36:09 rnnt_models:211] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-12-11 08:36:13 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /home/jovyan/work/HF_cache/hub/models--nvidia--stt_ru_conformer_transducer_large/snapshots/687d02db291e931455cf321abd625ef2b7f0b1a9/stt_ru_conformer_transducer_large.nemo.


In [5]:
# Модель для рескоринга bond005/ruT5-ASR и правки орфографии, а также функция для её использования
tokenizer_for_rescoring = T5Tokenizer.from_pretrained('bond005/ruT5-ASR')
model_for_rescoring = T5ForConditionalGeneration.from_pretrained('bond005/ruT5-ASR')
model_for_rescoring = model_for_rescoring.to(device_0)
    
def rescore(text: str, tokenizer: T5Tokenizer,
            model: T5ForConditionalGeneration) -> str:
    if len(text) == 0:  # if an input text is empty, then we return an empty text too
        return ''
    ru_letters = set('аоуыэяеёюибвгдйжзклмнпрстфхцчшщьъ')
    punct = set('.,:/\\?!()[]{};"\'-')
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    min_size = 3
    if x.input_ids.shape[1] <= min_size:
        return text  # we don't rescore a very short text
    out = model.generate(**x, do_sample=False, num_beams=5,
                         max_length=max_size, min_length=min_size)
    res = tokenizer.decode(out[0], skip_special_tokens=True).lower().strip()
    res = ' '.join(res.split())
    postprocessed = ''
    for cur in res:
        if cur.isspace() or (cur in punct):
            postprocessed += ' '
        elif cur in ru_letters:
            postprocessed += cur
    return (' '.join(postprocessed.strip().split())).replace('ё', 'е')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
# Модель для правки пунктуации от Сбера (опционально)
punct_model = SbertPuncCase()
punct_model = punct_model.to(device_0)

In [7]:
# LLM Saiga-Mistral и полезные функции для работы с этой моделью
MODEL_NAME = "IlyaGusev/saiga_mistral_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический медицинский ассистент. Ты выслушиваешь жалобы людей на проблемы со здоровьем и сообщаешь им предположительный диагноз"

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device_1 #"auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def step(inp, meta=''):
    if meta == '':
        system_prompt = DEFAULT_SYSTEM_PROMPT
    else:
        system_prompt =  meta
    сonversation = Conversation(system_prompt=system_prompt)   
    сonversation.add_user_message(inp)
    prompt = сonversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print("==============================")
    return(output)

In [9]:
# TeraTTS - русская модель синтеза речи
tts = TTS("TeraTTS/natasha-g2p-vits", add_time_to_end=1.0, tokenizer_load_dict=True) 

In [10]:
# Модель расстановки ударений
accentizer = RUAccent(workdir="./model")
accentizer.load(omograph_model_size='big_poetry', use_dictionary=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# TitaNet для эмбеддингов
titanet_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
titanet_model = titanet_model.to(device_0)

[NeMo I 2023-12-11 08:37:37 cloud:58] Found existing object /home/jovyan/.cache/torch/NeMo/NeMo_1.21.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-12-11 08:37:37 cloud:64] Re-using file from: /home/jovyan/.cache/torch/NeMo/NeMo_1.21.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-12-11 08:37:37 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-12-11 08:37:38 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-12-11 08:37:38 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-12-11 08:37:38 features:289] PADDING: 16
[NeMo I 2023-12-11 08:37:39 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/jovyan/.cache/torch/NeMo/NeMo_1.21.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.


In [12]:
# kNN для пола
sex_model = pickle.load(open('./weights/knn_sex_tn.pkl', 'rb'))

In [13]:
# модель для возраста
import torch.nn as nn
class FCN2(nn.Module):
    def __init__(self, num_classes=3):
        super(FCN2, self).__init__()

        self.fc1 = nn.Linear(192, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = self.dropout1(torch.relu(self.fc1(x)))
        x = self.dropout2(torch.sigmoid(self.fc2(x)))
        logits = self.fc(x)  
        return logits    

age_model = FCN2()
age_model.load_state_dict(torch.load('./weights/voice_to_age_tn_FCN2.pth'))
age_model = age_model.to(device_0)

### Обработка голосового запроса

Загрузка записи голосового запроса

In [14]:
# Нужно указать путь к wav. файлу с тестовым голосовым запросом
wav_file = './data/test_asr.wav'

Получение эмбеддинга с помощью TitaNet

In [15]:
%%time
tn_emb = titanet_model.get_embedding(wav_file).squeeze().detach().cpu().tolist()

CPU times: user 1.72 s, sys: 691 ms, total: 2.41 s
Wall time: 2.43 s


Определение пола

In [16]:
%%time
sex = sex_model.predict([tn_emb])[0]
sex

CPU times: user 980 ms, sys: 57.3 ms, total: 1.04 s
Wall time: 187 ms


1

In [17]:
# Мэппинг для запроса
if sex == 0:
    meta1 = 'женского пола'
else:
    meta1 = 'мужского пола'

Определение возрастного интервала

In [18]:
%%time
age_model.eval()
with torch.no_grad():
    inputs = torch.FloatTensor([tn_emb]).to(device_0)
    outputs = age_model(inputs)
    _, predicted = outputs.max(1)
    age = predicted[0].item()
age

CPU times: user 1.57 ms, sys: 2.27 ms, total: 3.85 ms
Wall time: 2.57 ms


1

In [19]:
# Мэппинг для запроса
if age == 0:
    meta2 = 'возрастом моложе 25 лет'
elif age == 1:
    meta2 = 'возрастом в пределах от 25 до 40 лет'
else:
    meta2 = 'возрастом старше 40 лет'

In [20]:
meta = f"Ты — Сайга, русскоязычный автоматический медицинский ассистент. Ты выслушиваешь жалобу пациента {meta1} {meta2} на проблемы со здоровьем и сообщаешь ему предположительный диагноз"

Расшифровка голоса в текст с помощью ru_conformer_transducer_large

In [21]:
%%time
text0 = asr_model.transcribe([wav_file])[0][0]
text0

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1.22 s, sys: 62 ms, total: 1.28 s
Wall time: 560 ms


'у меня поднялась температура заложила нос еще болит поясница'

Исправление орфографии с помощью ruT5-ASR

In [22]:
%%time
text1 = rescore(text0, tokenizer_for_rescoring, model_for_rescoring)
text1

CPU times: user 726 ms, sys: 11.8 ms, total: 738 ms
Wall time: 734 ms


'у меня поднялась температура заложило нос еще болит поясница'

Исправление пунктуации и заглавных букв с помощью SbertPuncCase

In [23]:
%%time
text2 = punct_model.punctuate(text1)
text2

CPU times: user 331 ms, sys: 39 ms, total: 370 ms
Wall time: 368 ms


'У меня поднялась температура, заложило нос, еще болит поясница.'

Передача вопроса в языковую модель

In [24]:
%%time
text3 = step(text2, meta)

У меня поднялась температура, заложило нос, еще болит поясница.
Ваша симптоматика может указывать на инфекционное заболевание, возможно, это грипп или общий респираторный вирус. Важно немедленно обратиться к врачу для получения лечения и консультации. Также важно придерживаться правил гигиены и избегать контакта с другими людьми, чтобы не передавать инфекцию.
CPU times: user 56.5 s, sys: 153 ms, total: 56.6 s
Wall time: 56.6 s


Расстановка ударений в ответе

In [25]:
%%time
text4 = accentizer.process_all(text3)
text4

CPU times: user 2.66 s, sys: 228 ms, total: 2.89 s
Wall time: 113 ms


'в+аша симптом+атика м+ожет ук+азывать н+а инфекци+онное заболев+ание, возм+ожно, +это гр+ипп +или +общий респират+орный в+ирус. в+ажно нем+едленно обрат+иться к врач+у дл+я получ+ения леч+ения и консульт+ации. т+акже в+ажно прид+ерживаться пр+авил гиги+ены и избег+ать конт+акта с друг+ими людьм+и, чт+обы н+е передав+ать инф+екцию.'

Синтез голосового ответа

In [26]:
%%time
# 'length_scale' можно использовать для замедления аудио для лучшего звучания (по умолчанию 1.1, указано здесь для примера)
audio = tts(text4, lenght_scale=2.0)  # Создать аудио. Можно добавить ударения, используя '+'

CPU times: user 3min 26s, sys: 2.61 s, total: 3min 29s
Wall time: 7.17 s


In [27]:
# Нужно указать путь, куда сохранять wav. файл с голосовым ответом
#tts.play_audio(audio)  # Воспроизвести созданное аудио
tts.save_wav(audio, "./reports/test_tts.wav")  # Сохранить аудио в файл